In [ ]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
lu1 = ee.ImageCollection('projects/wri-datalab/cities/urban_land_use/V1')
lu2 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V2')
lu = lu1.merge(lu2).select('lulc').mosaic()

In [ ]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020_v7')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)

In [ ]:
%%time
ato_res = defaultdict(list)
inf_res = defaultdict(list)
openspace_res = defaultdict(list)
nonres_res = defaultdict(list)
formal_res = defaultdict(list)
housingproj_res = defaultdict(list)
tot_res = defaultdict(list)

ato_total = []
inf_total = []
openspace_total = []
nonres_total = []
formal_total = []
housingproj_total = []
tot_total = []

scale = 10

for i in range(len(urbext_data_2020)):
    ua = urbext_data_2020.iloc[i]
    ua_f = urbext_2020.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
    geom = ua_f.geometry()
    lu_local_img = lu.clip(geom)
    lu_area_img = ee.Image.pixelArea().addBands(lu_local_img.select("lulc").reduce(ee.Reducer.firstNonNull()))
    #atomistic_img = lu_local_img.eq(2)
    #informal_img = lu_local_img.eq(3)
    #atomistic_area_img = lu_area_img.multiply(atomistic_img)
    #informal_area_img = lu_area_img.multiply(informal_img)
    #atomistic_area = atomistic_area_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area').getInfo()
    #informal_area = informal_area_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area').getInfo()
    def get_area(lu_class, too_big):
        if lu_class is not None:
            img = lu_local_img.eq(lu_class)
        else:
            img = lu_local_img.lte(5)
        area_img = lu_area_img.multiply(img)
        if not too_big:
            try:
                area = area_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area').getInfo()
            except:
                too_big = True
        if too_big:
            print(f'Tiling {lu_class}')
            def gridcell_area(f):
                geom = f.geometry()
                lu_local_img = lu.clip(geom)
                lu_area_img = ee.Image.pixelArea().addBands(lu_local_img.select("lulc").reduce(ee.Reducer.firstNonNull()))
                if lu_class is not None:
                    img = lu_local_img.eq(lu_class)
                else:
                    img = lu_local_img.lte(5)
                area_img = lu_area_img.multiply(img)
                return f.set('class_area', area_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area'))
            grid = ua_f.bounds().coveringGrid(proj=ua_f.geometry().projection().crs(), scale=10000).filterBounds(ua_f.geometry()).map(lambda f: f.intersection(ua_f.union(maxError=1).first(), maxError=1))
            area = grid.map(gridcell_area).aggregate_sum('class_area').getInfo()
        return area, too_big
    too_big = False
    openspace_area, too_big = get_area(0, too_big)
    nonres_area, too_big = get_area(1, too_big)
    atomistic_area, too_big = get_area(2, too_big)
    informal_area, too_big = get_area(3, too_big)
    formal_area, too_big = get_area(4, too_big)
    housingproj_area, too_big = get_area(5, too_big)
    total_area, too_big = get_area(None, too_big)
                
    openspace_res[ua['country']].append(openspace_area)
    nonres_res[ua['country']].append(nonres_area)
    ato_res[ua['country']].append(atomistic_area)
    inf_res[ua['country']].append(informal_area)
    formal_res[ua['country']].append(formal_area)
    housingproj_res[ua['country']].append(housingproj_area)
    tot_res[ua['country']].append(total_area)
    openspace_total.append(openspace_area)
    nonres_total.append(nonres_area)
    ato_total.append(atomistic_area)
    inf_total.append(informal_area)
    formal_total.append(formal_area)
    housingproj_total.append(housingproj_area)
    tot_total.append(total_area)
    
    print(i, end=' ')
    

In [ ]:
out_dicts = []

out_dicts.append(
    {
        'country': 'global',
        'urban_area_openspace_2020': sum(openspace_total),
        'urban_area_nonresidential_2020': sum(nonres_total),
        'urban_area_atomistic_subdivision_2020': sum(ato_total),
        'urban_area_informal_subdivision_2020': sum(inf_total),
        'urban_area_formal_subdivision_2020': sum(formal_total),
        'urban_area_housing_project_2020': sum(housingproj_total),
        'urban_area_total_2020': sum(tot_total),
        'fraction_area_openspace_2020': sum(openspace_total) / sum(tot_total),
        'fraction_area_nonresidential_2020': sum(nonres_total) / sum(tot_total),
        'fraction_area_atomistic_2020': sum(ato_total) / sum(tot_total),
        'fraction_area_informal_2020': sum(inf_total) / sum(tot_total),
        'fraction_area_informal_or_atomistic_2020': (sum(ato_total) + sum(inf_total)) / sum(tot_total),
        'fraction_area_formal_2020': sum(formal_total) / sum(tot_total),
        'fraction_area_housing_project_2020': sum(housingproj_total) / sum(tot_total)
    }
)

country_list = list(ato_res.keys())
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'urban_area_openspace_2020': sum(openspace_res[country]),
            'urban_area_nonresidential_2020': sum(nonres_res[country]),
            'urban_area_atomistic_subdivision_2020': sum(ato_res[country]),
            'urban_area_informal_subdivision_2020': sum(inf_res[country]),
            'urban_area_formal_subdivision_2020': sum(formal_res[country]),
            'urban_area_housing_project_2020': sum(housingproj_res[country]),
            'urban_area_total_2020': sum(tot_res[country]),
            'fraction_area_openspace_2020': sum(openspace_res[country]) / sum(tot_res[country]),
            'fraction_area_nonresidential_2020': sum(nonres_res[country]) / sum(tot_res[country]),
            'fraction_area_atomistic_2020': sum(ato_res[country]) / sum(tot_res[country]),
            'fraction_area_informal_2020': sum(inf_res[country]) / sum(tot_res[country]),
            'fraction_area_informal_or_atomistic_2020': (sum(ato_res[country]) + sum(inf_res[country])) / sum(tot_res[country]),
            'fraction_area_formal_2020': sum(formal_res[country]) / sum(tot_res[country]),
            'fraction_area_housing_project_2020': sum(housingproj_res[country]) / sum(tot_res[country])
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('LU_areas_10m.csv')